In [2]:
import os
import cv2
import zipfile
import shutil
import numpy as np
from IPython.display import FileLink

# Function to add PSF noise to an image
def add_psf_noise(image, psf_sigma):
    # (Same as previous code)
    psf_size = int(6 * psf_sigma)  # Adjust the size based on your requirements
    psf_kernel = cv2.getGaussianKernel(psf_size, psf_sigma)
    psf_kernel = psf_kernel @ psf_kernel.T  # 2D Gaussian kernel

    # Normalize the PSF kernel to ensure it sums to 1
    psf_kernel /= np.sum(psf_kernel)

    # Generate PSF noise
    psf_noise = np.random.normal(0, 1, image.shape)  # Generate random noise
    psf_noise = cv2.filter2D(psf_noise, -1, psf_kernel)  # Apply PSF kernel to the noise

    # Add PSF noise to the image
    noisy_image = image + psf_noise

    # Clip values to be within the valid range (0 to 255 for uint8 images)
    noisy_image = np.clip(noisy_image, 0, 255)

    return noisy_image.astype(np.uint8)

# Specify the parent folder containing subfolders
parent_folder = '/kaggle/input/201-hubble-galaxies/'

# Create the output directory if it does not exist
output_directory = '/kaggle/working/output'
os.makedirs(output_directory, exist_ok=True)

# Create a zip file to store the noisy images
zip_file_path = '/kaggle/working/noisy_images.zip'
zip_file_path = os.path.join('/kaggle/working/', zip_file_path)

with zipfile.ZipFile(zip_file_path, 'w') as zip_file:

    # List all subfolders in the parent folder
    subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

    # Loop through each subfolder
    for subfolder in subfolders:
        print(f"Processing images in {subfolder}...")

        # List all image files in the subfolder
        image_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.lower().endswith(('.jpg', '.png', '.jpeg'))]

        # Loop through each image file in the subfolder
        for image_path in image_files:
            print(f"Processing image: {image_path}")

            # Read the image
            original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Set the PSF sigma
            psf_sigma = 1.5  # Adjust as needed

            # Add PSF noise to the image
            noisy_image = add_psf_noise(original_image, psf_sigma)

            # Save the noisy image to a temporary file in the Kaggle working directory
            noisy_image_temp_path = '/kaggle/working/' + os.path.basename(image_path)
            cv2.imwrite(noisy_image_temp_path, noisy_image)

            # Add the noisy image to the zip file with a relative path
            relative_path = os.path.relpath(noisy_image_temp_path, '/kaggle/working')
            zip_file.write(noisy_image_temp_path, arcname=relative_path)

            # Remove the temporary noisy image file
            os.remove(noisy_image_temp_path)

            
shutil.make_archive("/kaggle/working/output",'zip',"/kaggle/working")    
FileLink(r'/kaggle/working/output.zip')
print("Processing complete.")
print(f"Noisy images are saved in the zip file: {zip_file_path}")


Processing images in /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92895cc48f_3...
Processing image: /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92895cc48f_3/the-grand-whirlpool-galaxy-m51-and-its-companion_40370191930_o.jpg
Processing image: /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92895cc48f_3/starburst-galaxy-ngc-3310-blazes-with-star-formation_27324165297_o.jpg
Processing image: /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92895cc48f_3/polar-ring-galaxy-ngc-4650a_28322199448_o.jpg
Processing image: /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92895cc48f_3/the-majestic-sombrero-galaxy-messier-104_42149708512_o.jpg
Processing image: /kaggle/input/201-hubble-galaxies/20240226f34ff26fc16d3af492c01db2de240b42d7d0a0a2eb38a7ceb834dc92